In [1]:
# Import libraries
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import requests
from bs4 import BeautifulSoup

C:\Users\Chris\AppData\Local\Temp\ipykernel_26076\894987728.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
# Functions
def scrape_batting_table(season_id):
    """Scrape the batting table from the CCBC website for a given season_id

    Args:
        season_id (str): The season_id to scrape the batting table for

    Returns:
        pd.DataFrame: The batting table for the given season_id
    """    
    # Create the path string using the season_id
    path = f'http://pointstreak.com/baseball/stats.html?{season_id}&view=teambatting'
    
    # Get the page
    page = requests.get(path)
    # Create the soup object
    soup = BeautifulSoup(page.content, 'html.parser')
    # Get the table
    table = soup.select('#bat_first > table:nth-child(1)')
    
    # Check if the table is empty
    if not table:
        print(f'No table found for season_id {season_id}')
        return None
    
    # Get the headers (th) from the table
    headers = [th.text.strip() for th in table[0].find_all('th')]
    # Add the season_id to the headers
    headers.append('season_id')
    
    # Create an empty list to store the rows
    rows = []
    
    # Get the rows (tr) from the table
    for tr in table[0].find_all('tr'):
        # Get the data (td) from the row
        row = [td.text.strip() for td in tr.find_all('td')]
        # Check if the row is empty if not append the season_id to the row and append the row to the rows list
        if row:
            row.append(season_id)
            rows.append(row)
    return pd.DataFrame(rows, columns=headers)

def scrape_team_tables(season_id):
    """Scrape the batting table from the CCBC website for a given season_id

    Args:
        season_id (str): The season_id to scrape the batting table for

    Returns:
        pd.DataFrame: The batting table for the given season_id
    """    
    # Create the path string using the season_id
    path = f'http://pointstreak.com/baseball/stats.html?{season_id}&view=teambatting'
    
    # Get the page
    page = requests.get(path)
    # Create the soup object
    soup = BeautifulSoup(page.content, 'html.parser')
    # Get the table
    table = soup.select('#bat_first > table:nth-child(1)')
    
    # Check if the table is empty
    if not table:
        print(f'No table found for season_id {season_id}')
        return None
    
    # Get the headers (th) from the table
    headers = [th.text.strip() for th in table[0].find_all('th')]
    # Add the season_id to the headers
    headers.append('season_id')
    
    # Create an empty list to store the rows
    rows = []
    
    # Get the rows (tr) from the table
    for tr in table[0].find_all('tr'):
        # Get the data (td) from the row
        row = [td.text.strip() for td in tr.find_all('td')]
        # Check if the row is empty if not append the season_id to the row and append the row to the rows list
        if row:
            row.append(season_id)
            rows.append(row)
    return pd.DataFrame(rows, columns=headers)



# %%
# Function to perform the scraping
def scrape_page(url, scrape_func):
    """Scrape the CCBC website for a given url and scrape_func

    Args:
        url (str): The url to scrape
        scrape_func (function): The function to use to scrape the table

    Returns:
        pd.DataFrame: The dataframe containing the scraped data
    """    
    
    # Get the page
    page = requests.get(url)
    # Create the soup object
    soup = BeautifulSoup(page.content, 'html.parser')
    # Get the select element with the id seasonid
    select = soup.find('select', {'id': 'seasonid'})
    # Get the options from the select element (this is the list of years)
    options = select.find_all('option')
    # Create a dictionary of the years (seasons) with the season_id as the key and the season name as the value
    seasons = {option['value']: option.text for option in options}
    print(seasons)
    
    # For season in season_ids scrape the table and append the results to a dataframe using the scrape_func
    df = pd.DataFrame()
    for season in seasons.keys():
        temp_df = scrape_func(season)
        # append the new row to the DataFrame
        df = pd.concat([df, temp_df])

    # Create a new variable called season that uses the season_id column to look up the season name in the seasons dictionary
    df['season'] = df['season_id'].map(seasons)

    # Drop the season_id column
    df.drop('season_id', axis=1, inplace=True)
    
    # Return the dataframe
    return df

def scrape_pitching_table(season_id):
    """Scrape the pitching table from the CCBC website for a given season_id

    Args:
        season_id (str): The season_id to scrape the pitching table for (this is the year)

    Returns:
        pd.DataFrame: The pitching table for the given season_id
    """    
    
    # Create the path string using the season_id
    path = f'http://pointstreak.com/baseball/stats.html?{season_id}&view=teampitching'

    # Get the page
    page = requests.get(path)
    # Create the soup object
    soup = BeautifulSoup(page.content, 'html.parser')
    # Get the table
    table = soup.select('#pitch_first > table:nth-child(1)')
    if not table:
        print(f'No table found for season_id {season_id}')
        return None

    # Get the headers from the table (th)
    headers = [th.text.strip() for th in table[0].find_all('th')]
    # Add the season_id to the headers
    headers.append('season_id')
    
    # Create an empty list to store the rows
    rows = []

    # Get the rows from the table (tr)    
    for tr in table[0].find_all('tr'):
        # Get the data from the row (td)
        row = [td.text.strip() for td in tr.find_all('td')]
        # Check if the row is empty if not append the season_id to the row and append the row to the rows list
        if row:
            row.append(season_id)
            rows.append(row)
    return pd.DataFrame(rows, columns=headers)

def scrape_standings_table(season_id):
    """Scrape the standings table from the CCBC website for a given season_id

    Args:
        season_id (str): The season_id to scrape the standings table for (this is the year)

    Returns:
        pd.DataFrame: The standings table for the given season_id
    """    
    # Create the path string using the season_id
    path = f'http://pointstreak.com/baseball/standings.html?{season_id}&stype=l'
    # Get the page
    page = requests.get(path)
    # Create the soup object
    soup = BeautifulSoup(page.content, 'html.parser')
    # Get the table
    table = soup.select('#psbb_standings > table:nth-child(1)')
    if not table:
        print(f'No table found for season_id {season_id}')
        return None

    # Get the headers from the table (th)
    headers = [th.text.strip() for th in table[0].find_all('th')]
    # Add the season_id to the headers
    headers.append('season_id')
    
    # Create an empty list to store the rows
    rows = []

    # Get the rows from the table (tr)    
    for tr in table[0].find_all('tr'):
        # Get the data from the row (td)
        row = [td.text.strip() for td in tr.find_all('td')]
        if row:
            row.append(season_id)
            rows.append(row)
    return pd.DataFrame(rows, columns=headers)

def scrape_qual_batters_table(season_id):
    """Scrape the qualified batters table from the CCBC website for a given season_id

    Args:
        season_id (str): The season_id to scrape the qualified batters table for (this is the year)

    Returns:
        pd.DataFrame: The qualified batters table for the given season_id
    """    
    
    # Create the path string using the season_id
    path = f'http://pointstreak.com/baseball/stats.html?{season_id}&view=batting'
    # Get the page
    page = requests.get(path)
    # Create the soup object
    soup = BeautifulSoup(page.content, 'html.parser')
    # Get the table
    table = soup.select('#battingresults')
    if not table:
        print(f'No table found for season_id {season_id}')
        return None

    # Get the headers from the table (th)
    headers = [th.text.strip() for th in table[0].find_all('th')]
    # Add the season_id to the headers
    headers.append('season_id')
    
    # Create an empty list to store the rows
    rows = []

    # Get the rows from the table (tr)    
    for tr in table[0].find_all('tr'):
        # Get the data from the row (td)
        row = [td.text.strip() for td in tr.find_all('td')]
        if row:
            row.append(season_id)
            rows.append(row)
    return pd.DataFrame(rows, columns=headers)

def scrape_qual_pitchers_table(season_id):
    """Scrape the qualified pitchers table from the CCBC website for a given season_id

    Args:
        season_id (str): The season_id to scrape the qualified pitchers table for (this is the year)

    Returns:
        pd.DataFrame: The qualified pitchers table for the given season_id
    """    
    
    # Create the path string using the season_id
    path = f'http://pointstreak.com/baseball/stats.html?{season_id}&view=pitching'
    # Get the page
    page = requests.get(path)
    # Create the soup object
    soup = BeautifulSoup(page.content, 'html.parser')
    # Get the table
    table = soup.select('#pitchingresults')
    if not table:
        print(f'No table found for season_id {season_id}')
        return None
    # Get the headers from the table (th)
    headers = [th.text.strip() for th in table[0].find_all('th')]
    # Add the season_id to the headers
    headers.append('season_id')
    
    # Create an empty list to store the rows
    rows = []

    # Get the rows from the table (tr)    
    for tr in table[0].find_all('tr'):
        # Get the data from the row (td)
        row = [td.text.strip() for td in tr.find_all('td')]
        if row:
            row.append(season_id)
            rows.append(row)
    return pd.DataFrame(rows, columns=headers)

In [8]:
def scrape_team_batting(season_id, team_id):
    """Scrape the team batting table from the CCBC website for a given season_id

    Args:
        season_id (str): The season_id to scrape the team batting table for (this is the year)
        team_id (str): The team_id to scrape the team batting table for

    Returns:
        pd.DataFrame: The team batting table for the given season_id
    """    
    
    # Create the path string using the season_id
    path = f'https://baseball.pointstreak.com/team_stats.html?teamid={team_id}&seasonid={season_id}'
    
    # Get the page
    page = requests.get(path)
    
    # Create the soup object
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # Find the table
    table = soup.find('table', class_ = 'table table-hover table-striped nova-stats-table nova-stats-table--fixed-first-col')

    # Fail Safe if no table is found
    if not table:
        print(f'No table found for season_id {season_id}')
        return None
    
    # Get the headers from the table (th)
    headers = [th.text.strip() for th in table.find_all('th')]
    
    # Add the season_id to the headers
    headers.append('season_id')
    
    # Create an empty list to store the rows
    rows = []

    # Get the rows from the table (tr)    
    for tr in table.find_all('tr'):
        # Get the data from the row (td)
        row = [td.text.strip() for td in tr.find_all('td')]
        if row:
            row.append(season_id)
            rows.append(row)
    return pd.DataFrame(rows, columns=headers)

season_id = '33653'
team_id = '142674'

test_df = scrape_team_batting(season_id, team_id)
test_df.head()

,Player,P,AVG,G,AB,R,H,2B,3B,HR,...,BB,HBP,SO,SF,SH,SB,CS,DP,E,season_id
0,"Morris, B",IF,.342,31,117,20,40,4,2,1,...,15,3,17,1,0,19,4,1,10,33653
1,"Robertson, J",OF,.333,3,3,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,33653
2,"Bourget, R",OF,.327,31,107,32,35,10,1,2,...,14,6,23,2,1,11,3,3,4,33653
3,"Popik, T",OF,.273,14,22,3,6,0,0,0,...,4,6,7,0,0,1,0,0,3,33653
4,"Browning, D",IF,.256,24,82,15,21,3,0,0,...,11,4,15,1,1,5,3,1,4,33653


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    nav = soup.find('ul', {'class': 'list-unstyled nav navbar-nav'})  # adjust this to find the correct list

    data = []
    for li in nav.find_all('li', {'class': 'nova-list-item'}):
        link = li.find('a', href=True)
        if 'Stats' in link.text:
            stats_url = 'https://baseball.pointstreak.com/' + link['href']
            stats_response = requests.get(stats_url)
            stats_soup = BeautifulSoup(stats_response.text, 'html.parser')
            # Scrape the stats and append them to data

    # df = pd.DataFrame(data)

    return None

urls = ['https://baseball.pointstreak.com/teamlist.html?leagueid=160&seasonid=33653']  # list of URLs to scrape
dfs = [scrape_page(url) for url in urls]
df = pd.concat(dfs, ignore_index=True)

In [1]:
import requests
from bs4 import BeautifulSoup

def print_team_names(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    team_list = soup.find('ul', {'class': 'ps_teamlist list-unstyled'})

    for li in team_list.find_all('li'):
        link = li.find('a', href=True)
        if link is not None:
            print(link.text)

print_team_names('https://baseball.pointstreak.com/teamlist.html?leagueid=160&seasonid=33653')

Edmonton Collegiate
Okanagan College
Prairie Baseball Academy
Thompson Rivers University
University of Calgary
University of Fraser Valley
Vancouver Island University
Victoria Collegiate


In [2]:
import requests
from bs4 import BeautifulSoup

def print_team_names(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    team_list = soup.find('ul', {'class': 'ps_teamlist list-unstyled'})

    for li in team_list.find_all('li'):
        link = li.find('a', href=True)
        if link is not None:
            team_url = 'https://baseball.pointstreak.com/' + link['href']
            print(f"Accessing page: {team_url}")
            team_response = requests.get(team_url)
            # You can add code here to do something with the team page
            print("Page accessed")

print_team_names('https://baseball.pointstreak.com/teamlist.html?leagueid=160&seasonid=33653')

Accessing page: https://baseball.pointstreak.com/team_home.html?teamid=142674&seasonid=33653
Page accessed
Accessing page: https://baseball.pointstreak.com/team_home.html?teamid=142669&seasonid=33653
Page accessed
Accessing page: https://baseball.pointstreak.com/team_home.html?teamid=142670&seasonid=33653
Page accessed
Accessing page: https://baseball.pointstreak.com/team_home.html?teamid=142668&seasonid=33653
Page accessed
Accessing page: https://baseball.pointstreak.com/team_home.html?teamid=142671&seasonid=33653
Page accessed
Accessing page: https://baseball.pointstreak.com/team_home.html?teamid=142672&seasonid=33653
Page accessed
Accessing page: https://baseball.pointstreak.com/team_home.html?teamid=142673&seasonid=33653
Page accessed
Accessing page: https://baseball.pointstreak.com/team_home.html?teamid=157388&seasonid=33653
Page accessed


In [5]:
import requests
from bs4 import BeautifulSoup

def print_team_names(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    team_list = soup.find('ul', {'class': 'ps_teamlist list-unstyled'})

    for li in team_list.find_all('li'):
        link = li.find('a', href=True)
        if link is not None:
            team_url = 'https://baseball.pointstreak.com/' + link['href']
            print(f"Accessing team page: {team_url}")
            team_response = requests.get(team_url)
            team_soup = BeautifulSoup(team_response.text, 'html.parser')
            nav = team_soup.find('ul', {'class': 'list-unstyled nav navbar-nav'})
            if nav is not None:
                stats_link = None
                for item in nav.find_all('li', {'class': 'nova-list-item'}):
                    link = item.find('a', href=True)
                    if link is not None and 'Stats' in link.text:
                        stats_link = link
                        break
                if stats_link is not None:
                    stats_url = 'https://baseball.pointstreak.com/' + stats_link['href']
                    print(f"Accessing stats page: {stats_url}")
                    stats_response = requests.get(stats_url)
                    print("Stats page accessed")

print_team_names('https://baseball.pointstreak.com/teamlist.html?leagueid=160&seasonid=33653')

Accessing team page: https://baseball.pointstreak.com/team_home.html?teamid=142674&seasonid=33653
Accessing stats page: https://baseball.pointstreak.com/team_stats.html?teamid=142674&seasonid=33653
Stats page accessed
Accessing team page: https://baseball.pointstreak.com/team_home.html?teamid=142669&seasonid=33653
Accessing stats page: https://baseball.pointstreak.com/team_stats.html?teamid=142669&seasonid=33653
Stats page accessed
Accessing team page: https://baseball.pointstreak.com/team_home.html?teamid=142670&seasonid=33653
Accessing stats page: https://baseball.pointstreak.com/team_stats.html?teamid=142670&seasonid=33653
Stats page accessed
Accessing team page: https://baseball.pointstreak.com/team_home.html?teamid=142668&seasonid=33653
Accessing stats page: https://baseball.pointstreak.com/team_stats.html?teamid=142668&seasonid=33653
Stats page accessed
Accessing team page: https://baseball.pointstreak.com/team_home.html?teamid=142671&seasonid=33653
Accessing stats page: https://

In [7]:
import requests
from bs4 import BeautifulSoup

def access_print_page(url):
    print_url = 'https://baseball.pointstreak.com/' + url
    print(f"Accessing print page: {print_url}")
    print_response = requests.get(print_url)
    print("Print page accessed")

def access_team_stats_pages(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    team_list = soup.find('ul', {'class': 'ps_teamlist list-unstyled'})

    for li in team_list.find_all('li'):
        link = li.find('a', href=True)
        if link is not None:
            team_url = 'https://baseball.pointstreak.com/' + link['href']
            print(f"Accessing team page: {team_url}")
            team_response = requests.get(team_url)
            team_soup = BeautifulSoup(team_response.text, 'html.parser')
            nav = team_soup.find('ul', {'class': 'list-unstyled nav navbar-nav'})
            if nav is not None:
                stats_link = None
                for item in nav.find_all('li', {'class': 'nova-list-item'}):
                    link = item.find('a', href=True)
                    if link is not None and 'Stats' in link.text:
                        stats_link = link
                        break
                if stats_link is not None:
                    stats_url = 'https://baseball.pointstreak.com/' + stats_link['href']
                    print(f"Accessing stats page: {stats_url}")
                    stats_response = requests.get(stats_url)
                    stats_soup = BeautifulSoup(stats_response.text, 'html.parser')
                    print_div = stats_soup.find('div', {'class': 'pull-left'})
                    print_link = print_div.find('a', href=True, string='Print')
                    if print_link is not None:
                        access_print_page(print_link['href'])

access_team_stats_pages('https://baseball.pointstreak.com/teamlist.html?leagueid=160&seasonid=33653')

Accessing team page: https://baseball.pointstreak.com/team_home.html?teamid=142674&seasonid=33653
Accessing stats page: https://baseball.pointstreak.com/team_stats.html?teamid=142674&seasonid=33653
Accessing print page: https://baseball.pointstreak.com/printstats.html?teamid=142674&seasonid=33653&view=batting
Print page accessed
Accessing team page: https://baseball.pointstreak.com/team_home.html?teamid=142669&seasonid=33653
Accessing stats page: https://baseball.pointstreak.com/team_stats.html?teamid=142669&seasonid=33653
Accessing print page: https://baseball.pointstreak.com/printstats.html?teamid=142669&seasonid=33653&view=batting
Print page accessed
Accessing team page: https://baseball.pointstreak.com/team_home.html?teamid=142670&seasonid=33653
Accessing stats page: https://baseball.pointstreak.com/team_stats.html?teamid=142670&seasonid=33653
Accessing print page: https://baseball.pointstreak.com/printstats.html?teamid=142670&seasonid=33653&view=batting
Print page accessed
Accessi

In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def access_print_page(url, h2_text):
    print_url = 'https://baseball.pointstreak.com/' + url
    print(f"Accessing print page: {print_url}")
    print_response = requests.get(print_url)
    dfs = pd.read_html(print_response.text, attrs={'border': '1'})
    if dfs:
        df = dfs[0]
        df.columns = df.iloc[0]
        df = df[1:]
        df.columns = df.columns.str.lower().str.replace(' ', '_')
        df = df[df.player != 'Totals']
        df_name = h2_text.lower().replace(' ', '_') + '_df'
        globals()[df_name] = df
        print(f"Data stored in DataFrame: {df_name}")

def access_team_stats_pages(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    team_list = soup.find('ul', {'class': 'ps_teamlist list-unstyled'})

    for li in team_list.find_all('li'):
        link = li.find('a', href=True)
        if link is not None:
            team_url = 'https://baseball.pointstreak.com/' + link['href']
            print(f"Accessing team page: {team_url}")
            team_response = requests.get(team_url)
            team_soup = BeautifulSoup(team_response.text, 'html.parser')
            nav = team_soup.find('ul', {'class': 'list-unstyled nav navbar-nav'})
            if nav is not None:
                stats_link = None
                for item in nav.find_all('li', {'class': 'nova-list-item'}):
                    link = item.find('a', href=True)
                    if link is not None and 'Stats' in link.text:
                        stats_link = link
                        break
                if stats_link is not None:
                    stats_url = 'https://baseball.pointstreak.com/' + stats_link['href']
                    print(f"Accessing stats page: {stats_url}")
                    stats_response = requests.get(stats_url)
                    stats_soup = BeautifulSoup(stats_response.text, 'html.parser')
                    print_div = stats_soup.find('div', {'class': 'pull-left'})
                    print_link = print_div.find('a', href=True, string='Print')
                    if print_link is not None:
                        h2_text = stats_soup.find('h2').text
                        access_print_page(print_link['href'], h2_text)

access_team_stats_pages('https://baseball.pointstreak.com/teamlist.html?leagueid=160&seasonid=33653')

Accessing team page: https://baseball.pointstreak.com/team_home.html?teamid=142674&seasonid=33653
Accessing stats page: https://baseball.pointstreak.com/team_stats.html?teamid=142674&seasonid=33653
Accessing print page: https://baseball.pointstreak.com/printstats.html?teamid=142674&seasonid=33653&view=batting


C:\Users\Chris\AppData\Local\Temp\ipykernel_7752\2912203201.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(print_response.text, attrs={'border': '1'})


Data stored in DataFrame: 

edmonton_collegiate


								division:_canadian_college_baseball_conf.edm _|_ league:_canadian_college_baseball_conference_(ccbc)ccbc


								season:_ccbc_2023
							
_df
Accessing team page: https://baseball.pointstreak.com/team_home.html?teamid=142669&seasonid=33653
Accessing stats page: https://baseball.pointstreak.com/team_stats.html?teamid=142669&seasonid=33653
Accessing print page: https://baseball.pointstreak.com/printstats.html?teamid=142669&seasonid=33653&view=batting


C:\Users\Chris\AppData\Local\Temp\ipykernel_7752\2912203201.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(print_response.text, attrs={'border': '1'})


Data stored in DataFrame: 

okanagan_college


								division:_canadian_college_baseball_conf.coyotes _|_ league:_canadian_college_baseball_conference_(ccbc)ccbc


								season:_ccbc_2023
							
_df
Accessing team page: https://baseball.pointstreak.com/team_home.html?teamid=142670&seasonid=33653
Accessing stats page: https://baseball.pointstreak.com/team_stats.html?teamid=142670&seasonid=33653
Accessing print page: https://baseball.pointstreak.com/printstats.html?teamid=142670&seasonid=33653&view=batting


...

C:\Users\Chris\AppData\Local\Temp\ipykernel_7752\2912203201.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(print_response.text, attrs={'border': '1'})


Data stored in DataFrame: 

university_of_fraser_valley


								division:_canadian_college_baseball_conf.cascades _|_ league:_canadian_college_baseball_conference_(ccbc)ccbc


								season:_ccbc_2023
							
_df
Accessing team page: https://baseball.pointstreak.com/team_home.html?teamid=142673&seasonid=33653
Accessing stats page: https://baseball.pointstreak.com/team_stats.html?teamid=142673&seasonid=33653
Accessing print page: https://baseball.pointstreak.com/printstats.html?teamid=142673&seasonid=33653&view=batting


C:\Users\Chris\AppData\Local\Temp\ipykernel_7752\2912203201.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(print_response.text, attrs={'border': '1'})


Data stored in DataFrame: 

vancouver_island_university


								division:_canadian_college_baseball_conf.mariners _|_ league:_canadian_college_baseball_conference_(ccbc)ccbc


								season:_ccbc_2023
							
_df
Accessing team page: https://baseball.pointstreak.com/team_home.html?teamid=157388&seasonid=33653
Accessing stats page: https://baseball.pointstreak.com/team_stats.html?teamid=157388&seasonid=33653
Accessing print page: https://baseball.pointstreak.com/printstats.html?teamid=157388&seasonid=33653&view=batting
Data stored in DataFrame: 

victoria_collegiate


								division:_canadian_college_baseball_conf.golden_tide _|_ league:_canadian_college_baseball_conference_(ccbc)ccbc


								season:_ccbc_2023
							
_df


C:\Users\Chris\AppData\Local\Temp\ipykernel_7752\2912203201.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(print_response.text, attrs={'border': '1'})
